In [ ]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('teatroSXXI.csv', sep='|')

# Mostrar las primeras filas
print(df.head())

In [ ]:
# Crear una copia del dataframe
df_procesado = df.copy()

# Reemplazar valores vacíos con 'nan'
df_procesado = df_procesado.fillna('nan')

# Añadir la nueva columna con fecha por defecto
df_procesado['fechaFormat'] = '0-00-00'

# Guardar el nuevo CSV
df_procesado.to_csv('teatro_fechaFormat.csv', index=False, sep='|')

# Mostrar las primeras filas del nuevo dataframe para verificar
print(df_procesado.head())

In [ ]:
%pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama
import pandas as pd
import time

# Inicializar Ollama
llm = Ollama(model="llama2")

# Leer el CSV
df = pd.read_csv('teatro_fechaFormat.csv', sep='|')

# Función para procesar una fecha con Ollama
def procesar_fecha(fecha):
    try:
        prompt = f"formatea esta fecha en el formato dd-mm-yyyy, dame solo la fecha formateada, no quiero explicaciones: {fecha}"
        respuesta = llm.invoke(prompt)
        return respuesta.strip()
    except Exception as e:
        print(f"Error procesando fecha: {fecha}. Error: {e}")
        return '00-00-0000'

# Procesar cada fecha y guardar inmediatamente
for idx, row in df.iterrows():
    fecha_original = row['Fecha de estreno']
    fecha_procesada = procesar_fecha(fecha_original)
    df.at[idx, 'fechaFormat'] = fecha_procesada
    # Imprimir información de cada fila
    print(f"Fila {idx}: Original: {fecha_original} -> Procesada: {fecha_procesada}")
    # Guardar después de cada actualización
    df.to_csv('teatro_fechaOllama.csv', index=False, sep='|')

print("Proceso completado!")

In [ ]:
import pandas as pd
import re

# Leer el CSV
df = pd.read_csv('teatro_fechaOllama.csv', sep='|')

# Función para validar el formato de fecha
def validar_formato_fecha(fecha):
    # Patrón para dd-mm-yyyy
    patron = r'^\d{2}-\d{2}-\d{4}$'
    if pd.isna(fecha) or not isinstance(fecha, str):
        return pd.NA
    return fecha if re.match(patron, fecha.strip()) else pd.NA

# Aplicar la validación a la columna fechaFormat
df['fechaFormat'] = df['fechaFormat'].apply(validar_formato_fecha)

# Guardar el resultado
df.to_csv('teatro_fechaOllama.csv', index=False, sep='|')

# Mostrar un resumen de los cambios
print("Muestra de los datos actualizados:")
print(df[['fechaFormat']].head())
print("\nCantidad de fechas válidas:", df['fechaFormat'].notna().sum())
print("Cantidad de fechas inválidas (NaN):", df['fechaFormat'].isna().sum())

In [ ]:
import pandas as pd
import re

# Leer el CSV
df = pd.read_csv('teatro_fechaOllama.csv', sep='|')

# Función para validar el formato de fecha
def validar_formato_fecha(fecha):
    # Patrón para dd-mm-yyyy
    patron = r'^\d{2}-\d{2}-\d{4}$'
    if pd.isna(fecha) or not isinstance(fecha, str):
        return pd.NA
    return fecha if re.match(patron, fecha.strip()) else pd.NA

# Aplicar la validación a la columna fechaFormat
df['fechaFormat'] = df['fechaFormat'].apply(validar_formato_fecha)

# Crear un nuevo DataFrame solo con las columnas deseadas
df_fechas = df[['Fecha de estreno', 'fechaFormat']]

# Guardar el resultado
df_fechas.to_csv('teatro_fechas_comparacion.csv', index=False, sep='|')

# Mostrar un resumen de los cambios
print("Muestra de los datos guardados:")
print(df_fechas.head())
print("\nCantidad de fechas válidas:", df['fechaFormat'].notna().sum())
print("Cantidad de fechas inválidas (NaN):", df['fechaFormat'].isna().sum())